In [1]:
import sys
print("=== Python version info ===\n{}".format(sys.version))

=== Python version info ===
3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]


In [2]:
import sqlite3 as db

# Connect to a database (or create one if it doesn't exist)
conn = db.connect('example.db')

In [3]:
# Create a 'cursor' for executing commands
c = conn.cursor()

In [4]:
# If this is not the first time you run this cell, 
# you need to delete the existed "Students" table first
c.execute("DROP TABLE IF EXISTS Students")

# create a table named "Students" with 2 columns: "gtid" and "name".
# the type for column "gtid" is integer and for "name" is text. 
c.execute("CREATE TABLE Students (gtid INTEGER, name TEXT)")

In [5]:
c.execute("INSERT INTO Students VALUES (123, 'Vuduc')")
c.execute("INSERT INTO Students VALUES (456, 'Chau')")
c.execute("INSERT INTO Students VALUES (381, 'Bader')")
c.execute("INSERT INTO Students VALUES (991, 'Sokol')")

In [6]:
conn.commit()

In [8]:
# An important (and secure!) idiom
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?)', more_students)
conn.commit()

In [9]:
c.execute("SELECT * FROM Students")
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Students:\n", results)

Your results: 14 
The entries of Students:
 [(123, 'Vuduc'), (456, 'Chau'), (381, 'Bader'), (991, 'Sokol'), (723, 'Rozga'), (882, 'Zha'), (401, 'Park'), (377, 'Vetter'), (904, 'Brown'), (723, 'Rozga'), (882, 'Zha'), (401, 'Park'), (377, 'Vetter'), (904, 'Brown')]


In [10]:
# Run this cell
c.execute('DROP TABLE IF EXISTS Takes')
c.execute('CREATE TABLE Takes (gtid INTEGER, course TEXT, grade REAL)')

In [14]:
gt_grades = [(123, "CSE 6040", 4.0), (123, "ISYE 6644", 3.0), (123, "MGMT 8803", 1.0), (991, "CSE 6040", 4.0), (991, "ISYE 6740", 4.0), (456, "CSE 6040", 4.0), (456, "ISYE 6740", 2.0), (456, "MGMT 8803", 3.0)]
c.executemany('INSERT INTO Takes VALUES (?, ?, ?)', gt_grades)
conn.commit()
# Displays the results of your code
c.execute('SELECT * FROM Takes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Takes:", results)

Your results: 8 
The entries of Takes: [(123, 'CSE 6040', 4.0), (123, 'ISYE 6644', 3.0), (123, 'MGMT 8803', 1.0), (991, 'CSE 6040', 4.0), (991, 'ISYE 6740', 4.0), (456, 'CSE 6040', 4.0), (456, 'ISYE 6740', 2.0), (456, 'MGMT 8803', 3.0)]


In [16]:
# See all (name, course, grade) tuples
query = '''
        SELECT Students.name, Takes.course, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid
'''

for match in c.execute(query): # Note this alternative idiom for iterating over query results
    print(match)

('Vuduc', 'CSE 6040', 4.0)
('Vuduc', 'ISYE 6644', 3.0)
('Vuduc', 'MGMT 8803', 1.0)
('Chau', 'CSE 6040', 4.0)
('Chau', 'ISYE 6740', 2.0)
('Chau', 'MGMT 8803', 3.0)
('Sokol', 'CSE 6040', 4.0)
('Sokol', 'ISYE 6740', 4.0)


In [19]:
query = '''
        SELECT Students.name, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid and Takes.course = "CSE 6040"
'''

c.execute(query)
results1 = c.fetchall()
results1

[('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

In [22]:
query = '''
        SELECT Students.name, Takes.grade
        FROM Students LEFT JOIN Takes ON
        Students.gtid = Takes.gtid
'''

# Executes your `query` string:
c.execute(query)
matches = c.fetchall()
for i, match in enumerate(matches):
    print(i, "->", match)

0 -> ('Vuduc', 1.0)
1 -> ('Vuduc', 3.0)
2 -> ('Vuduc', 4.0)
3 -> ('Chau', 2.0)
4 -> ('Chau', 3.0)
5 -> ('Chau', 4.0)
6 -> ('Bader', None)
7 -> ('Sokol', 4.0)
8 -> ('Sokol', 4.0)
9 -> ('Rozga', None)
10 -> ('Zha', None)
11 -> ('Park', None)
12 -> ('Vetter', None)
13 -> ('Brown', None)
14 -> ('Rozga', None)
15 -> ('Zha', None)
16 -> ('Park', None)
17 -> ('Vetter', None)
18 -> ('Brown', None)


In [24]:
query = '''
        SELECT gtid, AVG(grade)
        FROM Takes 
        GROUP BY gtid
'''

for match in c.execute(query):
    print(match)

(123, 2.6666666666666665)
(456, 3.0)
(991, 4.0)


In [25]:
c.close()
conn.close()